In [1]:
import torch
from torch.utils.data import DataLoader
from vit_pytorch import SimpleViT
from src.data.datasets import KilterDataset

In [2]:
val = KilterDataset(split="val")
train = KilterDataset(split="train")
test = KilterDataset(split="test")

In [8]:
dl = DataLoader(train, batch_size=16)

In [9]:
batch = next(iter(dl))

In [10]:
batch[0].size()

torch.Size([16, 4, 50, 50])

In [11]:
batch[1]

tensor([30, 40, 30, 50, 40, 45, 45, 40, 40, 45, 30, 50, 45, 40, 40, 40])